# Task 6: Final Model Evaluation

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset (Consider sampling to 100k rows)
df = pd.read_csv('Global_Health_Statistics.csv')
df_sample = df.sample(n=100000, random_state=42)  # Only 100k rows for now

# Define features and target
X = df_sample.drop(columns=['Mortality Rate (%)'])
y = df_sample['Mortality Rate (%)']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing pipelines
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('ordinal', OrdinalEncoder())])  # Faster than OneHotEncoder

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define optimized model
best_model = RandomForestRegressor(
    n_estimators=50,  # Reduced from 200
    max_depth=15,  
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    n_jobs=-1,  # Use all CPU cores
    random_state=42
)

# Create a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])

# Train the optimized model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Print results
print("Optimized Model Performance:")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")


Optimized Model Performance:
RMSE: 2.857311613574025
R² Score: -0.004097941180945641
